# Модуль 1.3 — OutputParser (PydanticOutputParser)

**Цель:** получать структурированный, типизированный вывод из модели и уметь обрабатывать ошибки.

**Что сделаем:**
- установим нужные библиотеки
- опишем Pydantic‑схему для данных
- заставим модель возвращать строго JSON по формату
- покажем безопасную обработку ошибок

In [ ]:
%pip -q install -U \
  langchain \
  langchain-openai \
  langchain-community \
  pydantic==2.12.3 \
  python-dotenv \
  tiktoken \
  requests==2.32.4

## Настройка ключа и base URL

Для AITunnel укажите `OPENAI_API_KEY` и `OPENAI_BASE_URL`. В Colab лучше хранить ключ в переменной окружения.

> Подсказка: можно задать ключ один раз в разделе **Secrets** (значок 🔑 слева в Colab), тогда он будет доступен во всех ячейках.

In [ ]:
import os
from getpass import getpass
from dotenv import load_dotenv

load_dotenv()

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("Введите OPENAI_API_KEY: ")

if not os.environ.get("OPENAI_BASE_URL"):
    os.environ["OPENAI_BASE_URL"] = "https://api.aitunnel.ru/v1/"

## Описание схемы и формата

Мы хотим получить строгое JSON‑описание товара. Опишем схему через Pydantic и попросим модель следовать ей.

**Почему это важно:**
- модель иногда «украшает» ответ, и парсер ломается
- строгий формат позволяет надежно интегрировать ответ в бэкенд

**Ключевые параметры:**
- `BaseModel` + `Field` — описываем типы и назначение полей
- `PydanticOutputParser` — превращает JSON в валидный объект и кидает ошибку, если формат нарушен

In [ ]:
from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser

class ProductInfo(BaseModel):
    name: str = Field(description="Название товара")
    price: float = Field(description="Цена в рублях")
    category: str = Field(description="Категория товара")

parser = PydanticOutputParser(pydantic_object=ProductInfo)
format_instructions = parser.get_format_instructions()
print(format_instructions)

## Запрос к модели с формат‑инструкциями

Мы добавим `format_instructions` прямо в промпт, чтобы модель вернула JSON строго по схеме.

**Практика:**
- вставляйте формат‑инструкции ближе к задаче
- явно просите только JSON без лишнего текста

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.messages import HumanMessage

base_url = os.environ.get("OPENAI_BASE_URL")
chat = ChatOpenAI(
    model="gpt-5.2-chat",
    temperature=0.2,
    max_tokens=256,
    base_url=base_url,
)

prompt = PromptTemplate(
    template=(
        "Извлеки информацию о товаре.\n"
        "{format_instructions}\n"
        "Текст: {text}"
    ),
    input_variables=["text"],
    partial_variables={"format_instructions": format_instructions},
)

text = "В каталоге появился новый товар: беспроводные наушники AirSound. Цена 4990 рублей, категория — электроника."
filled_prompt = prompt.format(text=text)

response = chat.invoke([HumanMessage(content=filled_prompt)])
print(response.content)

## Парсинг ответа и обработка ошибок

Мы валидируем ответ и получаем готовый объект `ProductInfo`. Если модель ошиблась — ловим исключение и можем сделать ретрай или показать ошибку пользователю.

In [ ]:
from langchain_core.exceptions import OutputParserException

try:
    product = parser.parse(response.content)
    print(product)
    print("Название:", product.name)
    print("Цена:", product.price)
    print("Категория:", product.category)
except OutputParserException as exc:
    print("Ошибка парсинга:", exc)
    # Здесь можно сделать повторный запрос с подсказкой или fallback на другую модель

## Практические рекомендации

- Всегда просите **только JSON** и повторяйте это в промпте.
- Снижайте `temperature` (0.1–0.3), когда важна структура.
- Логируйте сырой ответ модели для дебага.
- Если парсер падает — переформулируйте инструкцию и добавьте ретрай.
- Для продакшена стоит хранить и сырое сообщение, и распарсенный объект.